Implementación base
1. Entrenar un modelo QDA sobre el dataset *iris* utilizando las distribuciones *a priori* a continuación ¿Se observan diferencias?¿Por qué cree? _Pista: comparar con las distribuciones del dataset completo, **sin splitear**_.
    1. Uniforme (cada clase tiene probabilidad 1/3)
    2. Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones).


Modificar las probabilidades a priori para las clases afectará los errores de predicción en los datos de entrenamiento y de prueba. Esto se debe a como funcionan los clasificadores Bayesianos, que combinan las probabilidades a priori con las probabilidades condicionales para calcular las probabilidades a posteriori. Viendo la expresión de la clasificación Bayesiana:

Se basa en el Teorema de Bayes:

𝑃(𝐺=𝑔∣𝑋=𝑥)=𝑃(𝑋=𝑥∣𝐺=𝑔)𝑃(𝐺=𝑔)/𝑃(𝑋=𝑥)

Donde:

𝑃(𝐺=𝑔∣𝑋=𝑥) es la probabilidad a posteriori de la clase 𝑔 dado el dato 𝑥.

𝑃(𝑋=𝑥∣𝐺=𝑔) es la probabilidad condicional de 𝑥 dado que pertenece a la clase 𝑔.

𝑃(𝐺=𝑔) es la probabilidad a priori de la clase 𝑔.

𝑃(𝑋=𝑥 es la probabilidad marginal de 𝑥.

En un clasificador Bayesiano, la decisión se toma generalmente seleccionando la clase con la mayor probabilidad a posteriori. Esto implica que las probabilidades a priori
𝑃(𝐺=𝑔) juegan un papel crucial en la decisión final.

Impacto de las Probabilidades A Priori en los Errores de Predicción
Si inicialmente teeneos probabilidades a priori
[0.90,0.05,0.05] para las clases 1, 2 y 3, respectivamente, estamos asumiendo que la mayoría de los datos pertenecen a la clase 1. Esto influye en las predicciones del modelo, haciéndolo más inclinado a predecir la clase 1. Si la verdadera distribución de las clases es similar a esta, los errores de predicción en los datos de entrenamiento y en los datos de prueba (Ep) serán bajos.

Sin embargo, si cambias las probabilidades a priori a
[0.05,0.90,0.05, el modelo ahora asumirá que la mayoría de los datos pertenecen a la clase 2. Esto cambiará significativamente las decisiones del modelo, haciéndolo más inclinado a predecir la clase 2. Si la verdadera distribución de las clases no coincide con esta nueva suposición, los errores de predicción aumentarán.

El cambio en los errores de predicción ocurre porque las probabilidades a priori modifican las probabilidades a posteriori que usa el clasificador para tomar decisiones. Si las probabilidades a priori no reflejan la verdadera distribución de las clases, el clasificador tenderá a hacer predicciones incorrectas más frecuentemente.

Conclusión
Las probabilidades a priori influyen directamente en la probabilidad a posteriori calculada por el clasificador, y por ende, en las predicciones que hace el modelo. Si las probabilidades a priori están en línea con la verdadera distribución de las clases, los errores de predicción serán menores. Si no lo están, los errores de predicción aumentarán. Por lo tanto, es crucial estimar o definir correctamente las probabilidades a priori para lograr una buena performance del clasificador Bayesiano.

In [ ]:
# Se importan las librerias necesarias
import numpy as np
from numpy.linalg import det, inv

# Modelo

In [ ]:
class ClassEncoder:
  """
  Permite codificar etiquetas categóricas en valores numéricos y decodificarlos
  de vuelta a sus etiquetas originales.
  """

  def fit(self, y):
    """
    Ajusta el codificador a las etiquetas proporcionadas.
    paso a paso:
    np.unique(y) encuentra las etiquetas únicas en y y las almacena en self.names.
    Crea un diccionario self.name_to_class que asigna a cada etiqueta única un índice numérico.
    Almacena el tipo de datos de y en self.fmt.
    """
    self.names = np.unique(y)
    self.name_to_class = {name:idx for idx, name in enumerate(self.names)}
    self.fmt = y.dtype
    # Q1: por que no hace falta definir un class_to_name para el mapeo inverso?

  def _map_reshape(self, f, arr):
    """
    Descripción: Aplica una función f a cada elemento de un array arr y luego lo remodela a su forma original.
    paso a paso:
    arr.flatten() aplana el array a una dimensión.
    Aplica la función f a cada elemento del array aplanado.
    Convierte el resultado en un array de NumPy y lo remodela a la forma original de arr.
    """
    return np.array([f(elem) for elem in arr.flatten()]).reshape(arr.shape)
    # Q2: por que hace falta un reshape?

  def transform(self, y):
    """
    Transforma las etiquetas en números enteros usando el mapeo definido en fit.
    paso a paso:
    Utiliza _map_reshape para aplicar el diccionario self.name_to_class a cada etiqueta en y,
    convirtiéndolas en sus correspondientes valores numéricos.
    """
    return self._map_reshape(lambda name: self.name_to_class[name], y)

  def fit_transform(self, y):
    """
    Ajusta el codificador a las etiquetas y luego las transforma en una sola operación.
    paso a paso:
    Llama a fit(y) para ajustar el codificador.
    Luego llama a transform(y) para transformar las etiquetas ajustadas.
    """
    self.fit(y)
    return self.transform(y)

  def detransform(self, y_hat):
    """
    Convierte los valores numéricos de vuelta a sus etiquetas originales.
    paso a paso:
    Utiliza _map_reshape para aplicar self.names a cada índice en y_hat,
    convirtiéndolos de vuelta a sus etiquetas originales.
    """
    return self._map_reshape(lambda idx: self.names[idx], y_hat)

# **Ejercicio 1.1**

In [ ]:
class BaseBayesianClassifier:
  """
  Implementación base para un clasificador Bayesiano.
  Contiene métodos para ajustar el modelo y predecir clases basándose
  en probabilidades a priori y condicionales.
  """

  def __init__(self):
    """
    Inicializa el clasificador creando un objeto ClassEncoder para codificar
    etiquetas categóricas en números enteros.
    """
    self.encoder = ClassEncoder()

  def _estimate_a_priori(self, y):
    """
    Estima las probabilidades a priori para cada clase.
    paso a paso:
    np.bincount(y.flatten().astype(int)) cuenta el número de ocurrencias de cada valor entero en y.
    Divide por y.size para obtener las frecuencias relativas (probabilidades).
    Devuelve el logaritmo natural de estas probabilidades.
    """
    # Obtener el número de clases en y: 3 en este caso
    num_classes = len(np.unique(y))
    # Crear un array con probabilidades a priori iguales: 1/3 en este caso
    a_priori = np.full(num_classes, 1/num_classes)
    # Q3: para que sirve bincount?
    return np.log(a_priori)

  def _fit_params(self, X, y):
    """
    Método abstracto que debe ser implementado por una subclase para ajustar
    los parámetros del modelo específico.
    paso a paso:
    No tiene funcionalidad en esta clase base, solo establece que las subclases
    deben implementar este método.
    """
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    """
    Método abstracto que debe ser implementado por una subclase para predecir
    la probabilidad condicional logarítmica.
    paso a paso:
    No tiene funcionalidad en esta clase base, solo establece que las subclases
    deben implementar este método.
    """
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    """
    Ajusta el clasificador a los datos X y y.
    paso a paso:
    Codifica las etiquetas y usando ClassEncoder.
    Estima las probabilidades a priori si no se proporcionan.
    Verifica que las probabilidades a priori coincidan con el número de clases.
    Llama al método _fit_params para ajustar los parámetros del modelo específico.
    """
    # first encode the classes
    y = self.encoder.fit_transform(y)
    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)
    # check that a_priori has the correct number of classes
    assert len(self.log_a_priori) == len(self.encoder.names), "A priori probabilities do not match number of classes"
    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    # Q4: por que el _fit_params va al final? no se puede mover a, por ejemplo, antes de la priori?

  def predict(self, X):
    """
    Predice las etiquetas para las observaciones en X.
    paso a paso:
    Inicializa un array vacío y_hat para almacenar las predicciones.
    Itera sobre cada observación en X, predice su clase usando _predict_one,
    y almacena el resultado decodificado en y_hat.
    Devuelve y_hat como un vector fila.
    """
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=self.encoder.fmt)
    for i in range(m_obs):
      encoded_y_hat_i = self._predict_one(X[:,i].reshape(-1,1))
      y_hat[i] = self.encoder.names[encoded_y_hat_i]
    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    """
    Predice la clase para una única observación x.
    paso a paso:
    Calcula la probabilidad a posteriori logarítmica para cada clase sumando
    la probabilidad a priori logarítmica y la probabilidad condicional logarítmica.
    Devuelve el índice de la clase con la máxima probabilidad a posteriori.
    """
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                  in enumerate(self.log_a_priori) ]

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [ ]:
class QDA(BaseBayesianClassifier):
  """
  Clasifica los datos basándose en modelos Gaussianos con diferentes matrices
  de covarianza para cada clase.
  """

  def _fit_params(self, X, y):
    """
    flatten, se asegura que y sea un array unidimensional, lo cual es necesario
    para hacer comparaciones elementales como y == idx. Sin flatten,
    y == idx puede no funcionar correctamente si y tiene más de una dimensión.
    bias=True en np.cov ajusta el cálculo de la matriz de covarianza dividiendo
    por N (número de observaciones) en lugar de N-1. Esto proporciona una
    estimación de la covarianza basada en la población en lugar de la muestra.
    X es una matriz donde las filas representan características y las columnas
    representan observaciones. Calcular la media a lo largo de las columnas
    (axis=1) proporciona la media de cada característica para cada clase.
    """
    # estimate each covariance matrix
    self.inv_covs = [inv(np.cov(X[:,y.flatten()==idx], bias=True))
                      for idx in range(len(self.log_a_priori))]
    # Q5: por que hace falta el flatten y no se puede directamente X[:,y==idx]?
    # Q6: por que se usa bias=True en vez del default bias=False?
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]
    # Q7: que hace axis=1? por que no axis=0?

  def _predict_log_conditional(self, x, class_idx):
    """
    Predice el logaritmo de la probabilidad condicional de x dado class_idx.
    paso a paso:
    Obtiene la matriz de covarianza inversa para la clase class_idx.
    Calcula unbiased_x restando la media de la clase de x.
    Calcula el logaritmo de la probabilidad condicional usando la fórmula del
    modelo Gaussiano cuadrático.
    """
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

# Código para pruebas

Seteamos los datos

In [ ]:
# hiperparámetros
rng_seed = 2000

In [ ]:
from sklearn.datasets import load_iris, fetch_openml
"""
proporciona dos funciones para cargar conjuntos de datos: uno para el conjunto
de datos de Iris y otro para un conjunto de datos de pingüinos.
"""

def get_iris_dataset():
  """
  Carga el conjunto de datos de Iris.
  paso a paso:
  Utiliza load_iris de sklearn.datasets para cargar los datos.
  Extrae las características (X_full) y las etiquetas (y_full) del conjunto
  de datos cargado.
  Convierte las etiquetas numéricas a sus nombres correspondientes
  (e.g., de 0, 1, 2 a "setosa", "versicolor", "virginica").
  Devuelve las características y las etiquetas.
  """
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

def get_penguins():
  """
  Carga el conjunto de datos de pingüinos.
  paso a paso:
  Utiliza fetch_openml de sklearn.datasets para obtener el conjunto de datos de pingüinos.
  Selecciona las características y las etiquetas del conjunto de datos.
  Elimina columnas no numéricas ("island" y "sex").
  Elimina filas con valores faltantes.
  Devuelve las características y las etiquetas.
  """
  # get data
  df, tgt = fetch_openml(name="penguins", return_X_y=True, as_frame=True, parser='auto')
  # drop non-numeric columns
  df.drop(columns=["island","sex"], inplace=True)
  # drop rows with missing values
  mask = df.isna().sum(axis=1) == 0
  df = df[mask]
  tgt = tgt[mask]
  return df.values, tgt.to_numpy().reshape(-1,1)

# showing for iris
X_full, y_full = get_iris_dataset()

print(f"X: {X_full.shape}, Y:{y_full.shape}")

X: (150, 4), Y:(150, 1)


Separamos el dataset en train y test para medir performance

In [ ]:
# preparing data, train - test validation
# 70-30 split
from sklearn.model_selection import train_test_split

def split_transpose(X, y, test_sz, random_state):
  """
  Descripción: Divide los datos en conjuntos de train y test utilizando una
  división de 70-30 (train-test split), y luego transpone las matrices para que
  las observaciones sean columnas en lugar de filas.
  Parámetros:
  X: Matriz de características.
  y: Vector de etiquetas.
  test_sz: Tamaño del conjunto de test (proporción).
  random_state: Semilla aleatoria para reproducibilidad.
  Devuelve:
  X_train: Matriz de características de train transpuesta.
  y_train: Vector de etiquetas de train transpuesto.
  X_test: Matriz de características de test transpuesta.
  y_test: Vector de etiquetas de test transpuesto.
  """
  # split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)
  # transpose so observations are column vectors
  return X_train.T, y_train.T, X_test.T, y_test.T

def accuracy(y_true, y_pred):
  """
  Descripción: Calcula la precisión de las predicciones comparando las etiquetas
  verdaderas con las predichas.
  Parámetros:
  y_true: Vector de etiquetas verdaderas.
  y_pred: Vector de etiquetas predichas.
  Devuelve:
  La precisión de las predicciones.
  """
  return (y_true == y_pred).mean()

train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.3, rng_seed)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(4, 105) (1, 105) (4, 45) (1, 45)


Entrenamos un QDA y medimos su accuracy

In [ ]:
qda = QDA()

qda.fit(train_x, train_y)

In [ ]:
train_acc = accuracy(train_y, qda.predict(train_x))
test_acc = accuracy(test_y, qda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0190 while test error is 0.0667


# **Ejercicio 1.2**

In [ ]:
class BaseBayesianClassifier:
  """
  Implementación base para un clasificador Bayesiano.
  Contiene métodos para ajustar el modelo y predecir clases basándose
  en probabilidades a priori y condicionales.
  """

  def __init__(self):
    """
    Inicializa el clasificador creando un objeto ClassEncoder para codificar
    etiquetas categóricas en números enteros.
    """
    self.encoder = ClassEncoder()

  def _estimate_a_priori(self, y):
    """
    Estima las probabilidades a priori para cada clase.
    paso a paso:
    np.bincount(y.flatten().astype(int)) cuenta el número de ocurrencias de cada valor entero en y.
    Divide por y.size para obtener las frecuencias relativas (probabilidades).
    Devuelve el logaritmo natural de estas probabilidades.
    """
    # Obtener el número de clases en y: 3 en este caso
    num_classes = len(np.unique(y))
    # Crear un array con probabilidades a priori: 0.90, 0.05 y 0.05
    a_priori = np.array([0.05, 0.05, 0.90])
    print(a_priori)
    # Q3: para que sirve bincount?
    return np.log(a_priori)

  def _fit_params(self, X, y):
    """
    Método abstracto que debe ser implementado por una subclase para ajustar
    los parámetros del modelo específico.
    paso a paso:
    No tiene funcionalidad en esta clase base, solo establece que las subclases
    deben implementar este método.
    """
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    """
    Método abstracto que debe ser implementado por una subclase para predecir
    la probabilidad condicional logarítmica.
    paso a paso:
    No tiene funcionalidad en esta clase base, solo establece que las subclases
    deben implementar este método.
    """
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    """
    Ajusta el clasificador a los datos X y y.
    paso a paso:
    Codifica las etiquetas y usando ClassEncoder.
    Estima las probabilidades a priori si no se proporcionan.
    Verifica que las probabilidades a priori coincidan con el número de clases.
    Llama al método _fit_params para ajustar los parámetros del modelo específico.
    """
    # first encode the classes
    y = self.encoder.fit_transform(y)
    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)
    # check that a_priori has the correct number of classes
    assert len(self.log_a_priori) == len(self.encoder.names), "A priori probabilities do not match number of classes"
    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    # Q4: por que el _fit_params va al final? no se puede mover a, por ejemplo, antes de la priori?

  def predict(self, X):
    """
    Predice las etiquetas para las observaciones en X.
    paso a paso:
    Inicializa un array vacío y_hat para almacenar las predicciones.
    Itera sobre cada observación en X, predice su clase usando _predict_one,
    y almacena el resultado decodificado en y_hat.
    Devuelve y_hat como un vector fila.
    """
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=self.encoder.fmt)
    for i in range(m_obs):
      encoded_y_hat_i = self._predict_one(X[:,i].reshape(-1,1))
      y_hat[i] = self.encoder.names[encoded_y_hat_i]
    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    """
    Predice la clase para una única observación x.
    paso a paso:
    Calcula la probabilidad a posteriori logarítmica para cada clase sumando
    la probabilidad a priori logarítmica y la probabilidad condicional logarítmica.
    Devuelve el índice de la clase con la máxima probabilidad a posteriori.
    """
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                  in enumerate(self.log_a_priori) ]

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [ ]:
class QDA(BaseBayesianClassifier):
  """
  Clasifica los datos basándose en modelos Gaussianos con diferentes matrices
  de covarianza para cada clase.
  """

  def _fit_params(self, X, y):
    """
    flatten, se asegura que y sea un array unidimensional, lo cual es necesario
    para hacer comparaciones elementales como y == idx. Sin flatten,
    y == idx puede no funcionar correctamente si y tiene más de una dimensión.
    bias=True en np.cov ajusta el cálculo de la matriz de covarianza dividiendo
    por N (número de observaciones) en lugar de N-1. Esto proporciona una
    estimación de la covarianza basada en la población en lugar de la muestra.
    X es una matriz donde las filas representan características y las columnas
    representan observaciones. Calcular la media a lo largo de las columnas
    (axis=1) proporciona la media de cada característica para cada clase.
    """
    # estimate each covariance matrix
    self.inv_covs = [inv(np.cov(X[:,y.flatten()==idx], bias=True))
                      for idx in range(len(self.log_a_priori))]
    # Q5: por que hace falta el flatten y no se puede directamente X[:,y==idx]?
    # Q6: por que se usa bias=True en vez del default bias=False?
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]
    # Q7: que hace axis=1? por que no axis=0?

  def _predict_log_conditional(self, x, class_idx):
    """
    Predice el logaritmo de la probabilidad condicional de x dado class_idx.
    paso a paso:
    Obtiene la matriz de covarianza inversa para la clase class_idx.
    Calcula unbiased_x restando la media de la clase de x.
    Calcula el logaritmo de la probabilidad condicional usando la fórmula del
    modelo Gaussiano cuadrático.
    """
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

In [ ]:
qda = QDA()
qda.fit(train_x, train_y)

train_acc = accuracy(train_y, qda.predict(train_x))
test_acc = accuracy(test_y, qda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

[0.05 0.05 0.9 ]
Train (apparent) error is 0.0190 while test error is 0.1111
